# Ch1: Primitives

number of bits which are one

In [2]:
# No. of which are one
def count(x):
    j = 0
    while x:
        j += x & 1
        x >>= 1
    return j
count(12)

2

# Count Parity


Counting parity T(n) = O(n)

In [3]:
# count the parity
def parity(x):
    j = 0
    while x:
        j ^= x & 1
        x >>= 1
    return j
parity(7)

1

Counting Parity (more improved) T(n) = O(k)

In this we're clearing lowest set of bits(i.e. all bits till lowest 1) by performign x & (x-1).

e.g.:
1. 1100 & 1011 = 1000
2. 110101 & 110100 = 110100
   > 110100 & 110011 = 110000
   
   Like in above example we removed 1 from lsb then the next 1 from 4th position. and this is how we proceed. Ignoring all the zeroes decreases time.

In [1]:
def parity(x):
    j = 0
    while x:
        j ^= 1
        x &= x - 1
    return j
parity(12)

0

## Parity using lookup table (i.e. caching)
e.g. we can use lookup table for 2 bit words
all possible combination would be (00), (01), (10), (11). so our cache (lookup table) would look like (0, 1, 1, 0).
So, to compute parity of (11101010), we first shift given no. by 6 to get first two MSBs. (00000011) and use it in our lookup table to get it's parity. Then shift the given number by 4. We'll get (00001110) and then use mask (00000011) to get the next required two bits. Similarly, repeat the steps for other two 2-bit lookups.

---

So, we are shifting as follows:
shift by no.of bits in lookup table * 3
shift by no.of bits in lookup table * 2 & 000(NO.OFBITSINLOOKUPTABLE) MASK. in this case(00000011)
shift by no.of bits in lookup table & masking
no shifting & masking

---

Let n be the word size and L be the word size in cache.
Time complexity: T(n) = O(n/L)

In [ ]:
def parity(x):
    MASK_SIZE = 16
    BIT_MASK = 0xFFFF
    # here PRECOMPUTED_PARITY is lookup table
    return (
        PRECOMPUTED_PARITY[x >> (3 * MASK_SIZE)] ^
        PRECOMPUTED_PARITY[x >> (2 * MASK_SIZE) & BIT_MASK] ^
        PRECOMPUTED_PARITY[(x >> MASK_SIZE) & BIT_MASK] ^
        PRECOMPUTED_PARITY[x & BIT_MASK])

we can improve from worst case time complexity O(n) from previous algorithm using some XOR properties.
1. associative
2. commutative
3. XOR of a group of bits is its parity

e.g.
find parity of 11010111.
It's parity will be same as parity of (1101) Xor'd with (0111) which is (1010).
then parity of 1010 will be same as parity of (10) xor'd with (10). i.e. of (00)
now for last (00) = (0) xor'd with (0) i.e. 0.

Hence this method takes **T(n) = O(logn)**.

### Algorithm Intuition.
11010111 xor will be stored in last 4 bits.
11010111 ^ 00001101 and similarily on repeating we will get our parity on last bit.

In [1]:
def parity(x):
    x ^= x >> 32
    x ^= x >> 16
    x ^= x >> 8
    x ^= x >> 4
    x ^= x >> 2
    x ^= x >> 1
    return x & 0x1
parity(12)

0

# Swap Bits

Q. A 64-bit integer can be viewed as an array of 64bits, with the bit at index 0 corresponding to the
least significant bit (LSB), and the bit at index 63 corresponding to the most significant bit (MSB).
Implement code that takes as input a 64-bit integer and swaps the bits at indices i and j.

My Try with Py

In [41]:
def swap_bits(number, i, j):
    # make sure i is greater than j
    if (i < j):
        i, j = j, i
        
    # extracting ith and jth bits
    x = number & 2**i
    y = number & 2**j

    # if y is 0 then
    if (x ^ y == x):
        # placing 1
        number |= x >> (i-j)
        # removing 1
        number &= ~x
    
    # if x is 0 then
    if (x ^ y == y):
        # placing 1
        number |= y << (i-j)
        # removing 1
        number &= ~y
    
    return number

swap_bits(14, 0, 3)

7

Let's see a better approach

In [52]:
def swap_bits(number, i, j):
    # Extract ith and jth bits to see if they differ
    
    if (number >> i) & 1 != (number >> j) & 1:
        # ith and jth bits differ
        # Select bits to flip with bitmask
        bit_mask = (1 << i) | (1 << j)
        # here we just xor the bit mask bcz
        # 0 ^ 1 = 1
        # 1 ^ 1 = 0, we swapped here 0 and 1.
        number ^= bit_mask
    return number
swap_bits(14, 0, 3)

7

# Reverse Bits

Q. Write a program that takes a 64-bit unsigned integer and retums the 64-bit unsigned integer consisting of the bits of the input in reverse order. For example, if the input is (1110000000000001), the
output should be (1000000000000111).

My Try with Py (Brute-Force)

In [55]:
def reverse_bits(number):
    for i in range(2):
        number = swap_bits(number, i, 4-i-1)
    return number
reverse_bits(14)

7

### Using Lookup Table
make a lookup table such that for every 16-bit integer y, A[y] holds its bit reversal.
let number be divided into 4 16-bits numbers. y3, y2, y1, y0.
so,
* y3 is bit reversal of y0
* y2 is bit reversal of y1
* y1 is bit reversal of y2
* y0 is bit reversal of y3

**Time Complexity: T(n) = O(n/L)**
where, n = word size and L = word size of cache

In [ ]:
def reverse_bits(x):
    MASK_SIZE = 16
    BIT_MASK = 0xFFFF
    return (
        PRECOMPUTED_REVERSE[x & BIT_MASK] << (3 * MASK_SIZE) |
        PRECOMPUTED_REVERSE[(x >> MASK_SIZE) & BIT_MASK] << (2 * MASK_SIZE) |
        PRECOMPUTED_REVERSE[(x >> (2 * MASK_SIZE)) & BIT_MASK] << MASK_SIZE |
        PRECOMPUTED_REVERSE[(x >> (3 * MASK_SIZE)) & BIT_MASK])

# Find the closest integer with the same weight

In [9]:
def same_weight(n):
    def swap_bits(n, i, j):
        x, y = 1 << i, 1 << j
        if not (
            n & x == x and n & y == y or 
            n & x == 0 and n & y == 0
        ): return n ^ (x | y)
        return n
    count, n1 = 0, n
    temp = n & 1
    while n & 1 == temp:
        n >>= 1
        count += 1
    return swap_bits(n1, count, count - 1)
same_weight(6)

5

## better code

In [10]:
def same_weight(n):
    NUM_UNSIGNED_BITS = 64
    for i in range(NUM_UNSIGNED_BITS - 1):
        if (n >> i) & 1 != (n >> i + 1) & 1:
            return n ^ (1 << i | 1 << (i + 1)) # swap bits
    raise ValueError("every bit is same")
same_weight(6)

5

# Check if a decimal number is a palindrome

In [15]:
def is_palindrome(num):
    bits = 32
    num2 = num
    f, b = 0, num % 10
    while not f:
        f = num2 // 10**bits
        num2 %= 10**bits
        bits -= 1
    while f == b and num != 0:
        f = num2 // 10**bits
        num2 %= 10**bits
        num //= 10
        b = num % 10
        bits -= 1
    return f==b
is_palindrome(121)

True

In [22]:
def is_palindrome_number(x):
    import math
    if x <= 0:
        return x == 0
    
    nums_digit = math.floor(math.log10(x)) + 1
    msd_mask = 10 ** (nums_digit - 1)
    
    # we are going till half of number of digits
    for i in range(nums_digit//2):
        if x // msd_mask != x % 10:
            return False
        x %= msd_mask # removing MSD
        x //= 10      # removing LSD
        msd_mask //= 100 # as we are removing two digits that's why by 100 and not by 10
    return True
is_palindrome_number(15177151)

True

In [29]:
! git commit -m "decimal inte

[main 9dcc211] decimal integer palindrome
 3 files changed, 269 insertions(+), 16 deletions(-)
